# Init

In [1]:
import sys
sys.path.append(r"../../../pomegranate")
sys.path.append(r"../../../WhoCell/")
sys.path.append(r"../../../WhoCell/who_cell/")
sys.path.append(r"../../../WhoCell/who_cell/models/")
sys.path.append(r"../../../WhoCell/who_cell/simulation/")

In [2]:
from functools import reduce
from collections import Counter
import seaborn as sns

In [3]:
import pomegranate
from pomegranate import *
from pomegranate import HiddenMarkovModel
import numpy as np
import math 
import matplotlib.pyplot as plt
from functools import partial 
import itertools
import random
import copy
import pandas as pd
import itertools
import networkx as nx
import holoviews as hv
from IPython.display import display, HTML,clear_output
from scipy.stats import binom
from ast import literal_eval as make_tuple


from numba import jit
import numba

from gibbs_sampler import GibbsSampler
from simulation_for_gibbs import Simulator_for_Gibbs
from who_cell.experiments.experiment_report import ExperimentReport
from who_cell.experiments.gibbs_experiments import GibbsExperiment



from tqdm import tqdm
import snakeviz
%load_ext snakeviz
%load_ext autoreload


In [4]:
%connect_info

{
  "shell_port": 45971,
  "iopub_port": 39055,
  "stdin_port": 44663,
  "control_port": 51457,
  "hb_port": 47283,
  "ip": "127.0.0.1",
  "key": "0fb21d50-aa5ab25fc7db99f252518925",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-d1720709-7bb1-41ba-baab-e45696eadb83.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


# experiments

In [5]:
all_models_results = {}
er = ExperimentReport()

## simulate data 

### using known simulator for T

In [6]:
mutual_model_params_dict = {
            'is_acyclic': True,
            'known_mues': True,
            "is_few_observation_model": False,
            "is_only_seen": 'all',
            'N': 50,
            'd': 9,
            "bipartite": False,
            "inner_outer_trans_probs_ratio": 50,
            'n_states': 10,
            'sigma': 0.1,
            'number_of_smapled_traj': 1000,
            'p_prob_of_observation': 0.5,
            'N_itres': 10,
            'is_mh': False,
            'w_smapler_n_iter': 100}


In [7]:
def plot_results(transitions_results_list,experiments_name_list,transitions_probs,title):
    er = ExperimentReport()

    __l1_distance = lambda dist,state,known : abs(dist[state]-known) if state in dist.keys() else known
    __cross_entropy_distance = lambda dist,state,known : -1*known*np.log(dist[state]) if state in dist.keys() else (-1*known*np.log(0.0001))

    _l1_distance = lambda known_dist,comp_dist:sum(([__l1_distance(comp_dist,state,prob) for state,prob in known_dist.items()]))
    _cross_entropy_distance = lambda known_dist,comp_dist:sum([__cross_entropy_distance(comp_dist,state,prob) for state,prob in known_dist.items()])

    l1_distance = lambda known_trns,comp_trans : np.mean([_l1_distance(dist,comp_trans[state]) for state,dist in known_trns.items()])
    cross_entropy_distance = lambda known_trns,comp_trans : np.mean([_cross_entropy_distance(dist,comp_trans[state]) for state,dist in known_trns.items()])

    _trajectory_prob = lambda traj,model_trans:reduce(lambda x,y:x*y,[(model_trans[_f][_t] if _t in model_trans[_f].keys() else 0) for _f,_t in zip(traj,traj[1:])])


    all_l1_results = {}
    all_ce_results = {}
    for _trans_list,exp_name in zip(transitions_results_list,experiments_name_list) : 
        l1_dist = [l1_distance(transitions_probs,_t) for _t in _trans_list]
        ce_dist = [cross_entropy_distance(transitions_probs,_t) for _t in _trans_list]

        all_l1_results[exp_name] = l1_dist
        all_ce_results[exp_name] = ce_dist



#     for all_results,name in zip([all_l1_results,all_ce_results],['l1','crossEntropy']) : 
    for all_results,name in zip([all_l1_results],['l1']) : 
        fig, sub = plt.subplots(1,1,figsize=(8, 8))
        fig.suptitle(title)
        model_results_df = pd.DataFrame(all_results)

        sns.lineplot(data=model_results_df, ax=sub, legend='full', dashes=False)
        sub.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        sub.set_title(f"{name}")
        sub.set_xlabel("iter")
        sub.set_ylabel(f"{name}")

        plt.subplots_adjust(hspace=0.8)
        plt.subplots_adjust(wspace=0.8)

        plt.show()

#### build T

In [8]:
simulator = Simulator_for_Gibbs(mutual_model_params_dict['N'], mutual_model_params_dict['d'],
                                mutual_model_params_dict['n_states'], easy_mode=True,
                                max_number_of_sampled_traj =mutual_model_params_dict["number_of_smapled_traj"],
                                sigma=mutual_model_params_dict['sigma']) # we need max_number_of_sampled_traj to know how much traj to pre sample so the traj will be mutual

pome_results_1 = simulator.build_pome_model(mutual_model_params_dict['N'], mutual_model_params_dict['d'],
                               simulator.mues, simulator.sigmas)
simulator.update_known_mues_and_sigmes_to_state_mapping(pome_results_1["state_to_distrbution_param_mapping"])


ValueError: too many values to unpack (expected 2)

In [ ]:
def power_matrix_np(matrix,power):
    final = matrix
    for i in range(1,power):
        final = final.dot(matrix)
    return final

In [ ]:
_transition_matrix = pome_results_1['model'].dense_transition_matrix()

In [ ]:
n_states = mutual_model_params_dict['n_states']
_transition_matrix[0:n_states,0:n_states] = np.random.rand(n_states,n_states)


In [ ]:
_transition_matrix

In [ ]:
states = list(map(lambda s:(s.name if "-" not in s.name else s.name.split("-")[1]),pome_results_1['model'].states))
transition_matrix_df_1 = pd.DataFrame(data = _transition_matrix,columns = states,index=states)
transition_matrix_df_3 = pd.DataFrame(data = power_matrix_np(_transition_matrix,3),columns = states,index=states)
transition_matrix_df_10 =pd.DataFrame(data = power_matrix_np(_transition_matrix,10),columns = states,index=states)

In [ ]:
transition_matrix_df_3.loc['start'] = transition_matrix_df_1.loc['start']
transition_matrix_df_10.loc['start'] = transition_matrix_df_1.loc['start']

In [ ]:
# transition_matrix_np_1 = transition_matrix_df_1.values
# transition_matrix_np_3 = transition_matrix_df_3.values
# transition_matrix_np_10 = transition_matrix_df_10.values

In [ ]:
def reconstruction_test(transition_matrix_df,simulator,mutual_model_params_dict) :
    states = transition_matrix_df.columns.to_list()
    start_probs_dict = {k:v for k,v in transition_matrix_df.loc["start"].to_dict().items() if k not in ['start','end']}
    transition_matrix = transition_matrix_df.T.to_dict()
    transition_matrix_sparse = {k:{kk:vv for kk,vv in v.items() if vv != 0} for k,v in transition_matrix.items()}
    state_to_distrbution_param_mapping = {state:(eval(state) if state not in ['start','end'] else state) for state in states}
    
    model, all_model_pome_states = simulator._build_pome_model_from_params(state_to_distrbution_param_mapping,
                                                                  transition_matrix_sparse)
    
    (all_relvent_observations, _, _,_,known_w) = \
                simulator.simulate_observations(model,mutual_model_params_dict,'',from_pre_sampled_traj = False)
    
    sampler = GibbsSampler(mutual_model_params_dict['N'], mutual_model_params_dict['d'],'all')
    _, _, _, _, _, all_transitions_known_W = \
        sampler.sample_known_W(all_relvent_observations, start_probs_dict,
                       simulator.states_known_mues,simulator.states_known_sigmas,mutual_model_params_dict['N_itres'],known_w, w_smapler_n_iter=100,
                       is_mh=True,sample_missing_with_prior = False)
    
    _, _, _,_,_, all_transitions = \
        sampler.sample(all_relvent_observations, start_probs_dict,
                       simulator.states_known_mues,simulator.states_known_sigmas,mutual_model_params_dict['N_itres'], w_smapler_n_iter=100,
                       is_mh=True,N=2)

    
    transitions_results_list = [all_transitions_known_W,all_transitions]
    experiments_name_list = ["Known Gaps","Unknown Gaps"]
    
    plot_results(transitions_results_list,experiments_name_list,transition_matrix_sparse,"")


In [ ]:
# reconstruction_test(transition_matrix_df_1,simulator,mutual_model_params_dict)

In [ ]:
transition_matrix_df_1


In [ ]:
transition_matrix_df_3


In [ ]:
transition_matrix_df_10